# 理想解法 TOPSIS

xyfJASON

## 1 简述

设我们有 $n$ 个决策对象，每个对象有 $m$ 个指标（**已事先正向化处理**）。我们将各个指标中最好的数据合并起来得到**正理想解**，最差的数据合并起来得到 **负理想解**，计算每个决策对象到正理想解和负理想解的距离，既靠近正理想解又远离负理想解的就是最好的对象，并且可以据此对各个对象进行排序。

## 2 步骤

1. 设决策矩阵为 $(a_{ij})_{m\times n}$，即第 $i$ 个对象的第 $j$ 个指标为 $a_{ij}$（**已正向化**），对其进行向量规范化，得到规范化决策矩阵 $(b_{ij})_{m\times n}$：

   $$
   b_{ij}=\frac{a_{ij}}{\sqrt{\sum\limits_{i=1}^ma_{ij}^2}}
   $$


2. 构造加权规范化决策矩阵 $(c_{ij})_{m\times n}$：

   $$
   c_{ij}=w_j\cdot b_{ij}
   $$


3. 确定正理想解 $C^*$ 和负理想解 $C^0$：

   $$
   \begin{cases}
   c^*_j=\max\limits_{i=1}^m c_{ij}&j=1,2,\ldots,n\\
   c^0_j=\min\limits_{i=1}^m c_{ij}&j=1,2,\ldots,n
   \end{cases}
   $$


4. 计算各决策对象到正负理想解的距离（欧几里得距离）：

   $$
   \begin{cases}
   d^*_i=||c_i-C^*||_2=\sqrt{\sum\limits_{j=1}^n(c_{ij}-c_j^*)^2}\\
   d^0_i=||c_i-C^0||_2=\sqrt{\sum\limits_{j=1}^n(c_{ij}-c_j^0)^2}
   \end{cases}
   $$
   
   计算综合评价指标：
   
   $$
   f_i=\frac{d_i^0}{d_i^0+d_i^*}
   $$


5. 根据综合评价指标的大小从大到小排序，值越大，评价结果越好


## 3 代码模板

In [1]:
import numpy as np
import pandas as pd
from typing import Optional


class TOPSIS:
    def __init__(self,
                 x: np.ndarray,
                 weights: Optional[np.ndarray] = None) -> None:
        assert len(x.shape) == 2 and (weights is None or len(weights.shape) == 1)
        assert (x >= 0).all() and (x <= 1).all()
        assert weights is None or weights.shape[0] == x.shape[1]
        self.x = x
        self.weights = np.ones(x.shape[1]) if weights is None else weights

    def run(self) -> np.ndarray:
        b = self.x / np.sqrt(np.sum(self.x ** 2, axis=0))
        c = self.weights * b
        c_pos = c.max(axis=0)
        c_neg = c.min(axis=0)
        d_pos = np.sqrt(np.sum((c - c_pos) ** 2, axis=1))
        d_neg = np.sqrt(np.sum((c - c_neg) ** 2, axis=1))
        f = d_neg / (d_neg + d_pos)
        return f

## 4 实例

5 所研究生院的相关数据资料如下：

In [2]:
df = pd.DataFrame([[0.1, 5, 5000, 4.7],
                   [0.2, 6, 6000, 5.6],
                   [0.4, 7, 7000, 6.7],
                   [0.9, 10, 10000, 2.3],
                   [1.2, 2, 400, 1.8]])
df.columns = ['人均专著', '生师比', '科研经费', '逾期毕业率']
df.index = list('12345')
df

,人均专著,生师比,科研经费,逾期毕业率
1,0.1,5,5000,4.7
2,0.2,6,6000,5.6
3,0.4,7,7000,6.7
4,0.9,10,10000,2.3
5,1.2,2,400,1.8


其中人均专著、科研经费是极大型数据，逾期毕业率是极小型数据，生师比是区间型数据，最佳区间为 $[5,6]$，下面对数据进行正向化处理

In [3]:
from PositiveScaler import positive_scale
kind = np.array([0, 3, 0, 1])
interval_best = np.zeros((4, 2))
interval_best[1, :] = np.array([5, 6])
df.iloc[:, :] = positive_scale(x=df.values, kind=kind, interval_best=interval_best)
df

,人均专著,生师比,科研经费,逾期毕业率
1,0.000000,1.000000e+00,0.479167,0.408163
2,0.090909,1.000000e+00,0.583333,0.224490
3,0.272727,7.500000e-01,0.687500,0.000000
4,0.727273,2.500222e-13,1.000000,0.897959
5,1.000000,2.500000e-01,0.000000,1.000000


取权重为 $[0.2, 0.3, 0.4, 0.1]$，按 TOPSIS 法计算最终的指标

In [4]:
solver = TOPSIS(x=df.values, weights=np.array([.2, .3, .4, .1]))
res = solver.run()
res

array([0.5131253 , 0.5626838 , 0.59095594, 0.61892515, 0.36374483])

由大到小排序，可知研究生院从优到劣的次序为 4,3,2,1,5